# Основные точки руки

## Hands модель - множество рук
Данная модель позволяет находить множество рук но не различает правую и левую руки:

In [ ]:
#pip install opencv-python mediapipe msvc-runtime
import mediapipe as mp
import time
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt

Определение поднят большой палец вверх или опущен вниз для всех имеющихся рук на видео:

In [5]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Инициализация Mediapipe Hands
hands = mp_hands.Hands(max_num_hands=3, min_detection_confidence=0.7,
                           min_tracking_confidence=0.5,)

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while cap.isOpened():
    ret, image = cap.read()
    if not ret:
        break

    # Перевод кадра в RGB
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Обнаружение рук на кадре
    results = hands.process(rgb_image)

    # Отрисовка результатов на кадре
    if results.multi_hand_landmarks:
        for i, hand_landmarks in enumerate(results.multi_hand_landmarks):
            # Отрисовка точек руки
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Позиция большого пальца (4-й элемент) правой руки
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            thumb_tip_x, thumb_tip_y = int(thumb_tip.x * image.shape[1]), int(thumb_tip.y * image.shape[0])

            # Позиция всех кончиков пальцев правой руки кроме 4 - большого
            fingertips = [hand_landmarks.landmark[i] for i in range(1, 4)]  
            fingertips_x = [int(point.x * image.shape[1]) for point in fingertips]
            fingertips_y = [int(point.y * image.shape[0]) for point in fingertips]

            # Поднят ли большой палец
            thumb_up = thumb_tip_y < min(fingertips_y)

            # Отрисовка символа класса
            class_symbol = ':)' if thumb_up else ':('
            cv2.putText(image, class_symbol, (50, (i*70) + 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 100, 0), 2, cv2.LINE_AA)


    # Отображение кадра
    cv2.imshow('Hand Tracking', image)

    # Выход из цикла по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()


## Holistic модель - различаем праую и лк

## Holistic модель - сразу все находим (руки, тело и лицо)

In [ ]:
# Grabbing the Holistic Model from Mediapipe and
# Initializing the Model
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
	min_detection_confidence=0.5,
	min_tracking_confidence=0.5
)


# Initializing the drawing utils for drawing the facial landmarks on image
mp_drawing = mp.solutions.drawing_utils


drawing_spec_lines_r = mp_drawing.DrawingSpec(  # линии соединения
          color=(255,0,0),
          thickness=2,
          circle_radius=1
        )

drawing_spec_lines_l = mp_drawing.DrawingSpec(  # линии соединения
          color=(0,255,0),
          thickness=2,
          circle_radius=1
        )

# (0) in VideoCapture is used to connect to your computer's default camera
capture = cv2.VideoCapture(0)
 
# Initializing current time and precious time for calculating the FPS
previousTime = 0
currentTime = 0
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
 
while capture.isOpened():
    # capture frame by frame
    ret, frame = capture.read()
 
    # Converting the from BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
 
    # Making predictions using holistic model
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = holistic_model.process(image)
    image.flags.writeable = True
 
    # Converting back the RGB image to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
 
    # Drawing the Facial Landmarks
    mp_drawing.draw_landmarks(
      image,
      results.face_landmarks,
      mp_holistic.FACEMESH_CONTOURS,
      mp_drawing.DrawingSpec(
        color=(255,0,255),
        thickness=1,
        circle_radius=1
      ),
      mp_drawing.DrawingSpec(
        color=(0,255,255),
        thickness=1,
        circle_radius=1
      )
    )
 
    # Drawing Right hand Land Marks
    mp_drawing.draw_landmarks(
      image, 
      results.right_hand_landmarks, 
      mp_holistic.HAND_CONNECTIONS,
      connection_drawing_spec=drawing_spec_lines_r
    )
 
    # Drawing Left hand Land Marks
    mp_drawing.draw_landmarks(
      image, 
      results.left_hand_landmarks, 
      mp_holistic.HAND_CONNECTIONS,
      connection_drawing_spec=drawing_spec_lines_l
    )

    # Drawing pose Marks
    mp_drawing.draw_landmarks(
      image, 
      results.pose_landmarks, 
      mp_holistic.POSE_CONNECTIONS
    )

     
    # Calculating the FPS
    currentTime = time.time()
    fps = 1 / (currentTime-previousTime)
    previousTime = currentTime
     
    # Displaying FPS on the image
    cv2.putText(image, str(int(fps))+" FPS", (10, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
 
    # Display the resulting image
    cv2.imshow("Facial and Hand Landmarks", image)
 
    # Enter key 'q' to break the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
 
# When all the process is done
# Release the capture and destroy all windows
capture.release()
cv2.destroyAllWindows()